## How to solve least absolute error regression using linear programming

This page explains how to translate the least absolute error regression problem into a linear programming problem in standard form.
This amounts to a proof of the correctness of the `l1_fit`
function.

### The LAE Problem

Formulate the least absolute error
problem (LAE) as follows: 
find vector $\mathbf{m}$ of dimension $d$ and scalar $k$
which minimizes the ${\lVert \mathbf{e} \rVert}_1$ in the equation (1)
$$
\mathbf{U}  \cdot \mathbf{m} + k = \mathbf{v} + \mathbf{e}
$$
Where $\mathbf{U}$ is a given $n$ by $d$ matrix and $v$ is a
given vector
of length $n$.

Please see
[./1_Notebooks as Notes.ipynb](./1_Notebooks as Notes.ipynb)
for LAE examples in one and two dimensions with discussion.

### The LP Problem

We must translate the LAE problem into the standard linear programming
problem (LP).

$$ \underset{\mathbf{x}}{\text{minimize}} \;
   \mathbf{c} \cdot \mathbf{x}
$$
subject to constraints
$$
   \mathbf{A} \cdot \mathbf{x} \leq \mathbf{b}
$$
Where $\mathbf{A}$ is a given matrix, $\mathbf{b}$
is a given vector, and $\mathbf{x}$
is a variable vector of suitable dimensions.

We use the translation described below to call `scipy.optimize.linprog`
to solve the LAE problem as a LP problem and extract the results
from the LP formulation.
In the case of the `scipy.optimize.linprog` implementation
we will 
also need to specify
`bounds` (or lack of them) on $\mathbf{x}$ components
since some of the components are not bounded.

### The translation

Express ${\lVert \mathbf{e} \rVert}_1$ using equation (1) as
$$
{\lVert \mathbf{e} \rVert}_1
=
 {\lVert (\mathbf{U}  \cdot \mathbf{m} + k) - \mathbf{v} \rVert}_1
$$
If we switch variables to non-negative $\mathbf{e'} = | \mathbf{e} |$ then
${\lVert \mathbf{e} \rVert}_1 = {\lVert \mathbf{e'} \rVert}_1$
and equation LAE (1) is equivalent to (2)
$$ \underset{\mathbf{e'}}{\text{minimize}} \;
   {\lVert \mathbf{e'} \rVert}_1
$$
subject to constraints that the absolute residuals $\mathbf{e'}$
are larger than the positive or negative differences between
the observed and predicted values of the regression:
$$
   \mathbf{e'} \geq ((\mathbf{U}  \cdot \mathbf{m} + k) - \mathbf{v})
$$
$$
   \mathbf{e'} \geq -((\mathbf{U}  \cdot \mathbf{m} + k) - \mathbf{v})
$$
where
$$ 
   {\lVert \mathbf{e'} \rVert}_1 = \sum e'_i
$$
because the $e_i$ are non-negative.

Rewrite (2) as (3)
$$ \underset{\mathbf{e'}}{\text{minimize}} \;
   \sum e'_i
$$
subject to the constraints
$$
   \mathbf{v} \geq ((\mathbf{U}  \cdot \mathbf{m} + k)) - \mathbf{e'}
$$
$$
   -\mathbf{v} \geq -((\mathbf{U}  \cdot \mathbf{m} + k)) - \mathbf{e'}
$$

The above (3) is recognizable as a linear programming problem statement.
To get the statement in standard form define variable vector
$$
\mathbf{x} = 
\begin{bmatrix} 
e'_1 \\
e'_2 \\
\vdots\\
e'_n\\
m_1\\
m_2\\
\vdots\\
m_d\\
k \\
\end{bmatrix}
$$
with coefficient vector
$$
\mathbf{c} =  [ \underset{n}{1, \dots 1,} \underset{d}{0, \dots 0,} 0 ]
$$
and we have
$$
 \sum e'_i = \mathbf{c} \cdot \mathbf{x}
$$
as the objective function to minimize.

Furthermore we may write the constraints (3) as
$$
\begin{bmatrix}
-1& 0 & \dots & 0 & u_{1,1} & \dots & u_{1,d} & 1\\
0 & -1 & \dots & 0 & u_{2,1} & \dots & u_{1,d} & 1\\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots\\
0 & 0 & \dots & -1 & u_{n,1} & \dots & u_{n,d} & 1\\
-1& 0 & \dots & 0 & -u_{1,1} & \dots & -u_{1,d} & -1\\
0 & -1 & \dots & 0 & -u_{2,1} & \dots & -u_{1,d} & -1\\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
0 & 0 & \dots & -1 & -u_{n,1} & \dots & -u_{n,d} & -1\\
\end{bmatrix}
\cdot
\begin{bmatrix} 
e'_1 \\
e'_2 \\
\vdots\\
e'_n\\
m_1\\
m_2\\
\vdots\\
m_d\\
k \\
\end{bmatrix}
   \leq
\begin{bmatrix} 
v_1 \\
v_2 \\
\vdots\\
v_n\\
-v_1\\
-v_2\\
\vdots\\
-v_n\\
\end{bmatrix}
$$

Translating to matrix notation if we set
$$
\mathbf{A} = 
\begin{bmatrix}
    -\mathbf{I}_n & \mathbf{U} & \mathbf{1}_n  \\
    -\mathbf{I}_n & -\mathbf{U} & -\mathbf{1}_n  \\
\end{bmatrix}
$$
$$
\mathbf{b} = 
\begin{bmatrix}
    \mathbf{v} \\
    -\mathbf{v} \\
\end{bmatrix}
$$
Then we have translated (3) to the standard LP problem

$$ \underset{\mathbf{x}}{\text{minimize}} \;
   \mathbf{c} \cdot \mathbf{x}
$$
subject to constraints
$$
   \mathbf{A} \cdot \mathbf{x} \leq \mathbf{b}
$$
as desired where $\mathbf{A}$ is a matrix of dimension
$2n$ by $n + d + 1$ and the `l1_fit` function also specifies
that the $\mathbf{e}$ components of $\mathbf{x}$ are
non-negative whereas the $\mathbf{m}$ and $k$ components
are unbounded.